In [36]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [37]:
raw=pd.read_csv('ap_dataset_labeled.csv')
raw


,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,channel
0,0.0,-0.002432,-0.044127,-0.168850,-0.404639,-0.773205,-1.290878,-1.967325,-2.804175,-3.795185,...,1.060049e-03,1.055896e-03,1.051771e-03,1.047676e-03,1.043609e-03,0.001040,0.001036,0.001032,0.001028,GNa
1,0.0,-0.002408,-0.043682,-0.167142,-0.400537,-0.765338,-1.277670,-1.947037,-2.774963,-3.755146,...,8.714939e-04,8.683297e-04,8.651874e-04,8.620671e-04,8.589685e-04,0.000856,0.000853,0.000850,0.000847,GNa
2,0.0,-0.002385,-0.043237,-0.165435,-0.396436,-0.757471,-1.264464,-1.926759,-2.745770,-3.715146,...,7.552828e-04,7.527204e-04,7.501869e-04,7.476833e-04,7.452114e-04,0.000743,0.000740,0.000738,0.000736,GNa
3,0.0,-0.002301,-0.042753,-0.163701,-0.392315,-0.749590,-1.251251,-1.906479,-2.716589,-3.675181,...,6.761228e-04,6.741544e-04,6.721797e-04,6.702074e-04,6.682260e-04,0.000666,0.000664,0.000662,0.000660,GNa
4,0.0,-0.002338,-0.042347,-0.162020,-0.388233,-0.741742,-1.238065,-1.886229,-2.687443,-3.635262,...,6.132600e-04,6.116717e-04,6.102570e-04,6.090416e-04,6.080535e-04,0.000607,0.000607,0.000607,0.000605,GNa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,0.0,1.098678,2.286103,2.583387,2.690380,2.777422,2.865207,2.936723,2.969902,2.948981,...,1.021853e-06,1.155975e-06,1.242497e-06,1.333977e-06,1.424657e-06,0.000002,0.000002,0.000002,0.000002,GpCa
1976,0.0,1.109934,2.307669,2.606190,2.713114,2.800222,2.888272,2.960047,2.993270,2.972047,...,1.347513e-06,1.331165e-06,1.320712e-06,1.316883e-06,1.320442e-06,0.000001,0.000001,0.000001,0.000001,GpCa
1977,0.0,1.121183,2.329188,2.628915,2.735754,2.822920,2.911228,2.983256,3.016522,2.994997,...,1.509715e-06,1.511124e-06,1.515017e-06,1.521571e-06,1.530969e-06,0.000002,0.000002,0.000002,0.000001,GpCa
1978,0.0,1.132426,2.350658,2.651563,2.758301,2.845515,2.934075,3.006353,3.039659,3.017833,...,7.954999e-07,7.776499e-07,7.985281e-07,8.632145e-07,9.769840e-07,0.000001,0.000001,0.000001,0.000001,GpCa


In [38]:


X= raw.drop('channel', axis=1).values.astype(np.float32)  # shape: (1980, 501)
labels = raw['channel'].values.tolist()                    # shape: (1980,)
unique_labels = sorted(set(labels))
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}
y = np.array([label_to_int[label] for label in labels], dtype=np.int32)


In [39]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [40]:


BATCH_SIZE = 32

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)


In [41]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(501,)),              # Input layer (AP waveform)
    tf.keras.layers.Dense(130, activation='relu'),    # 🔥 One hidden layer
    tf.keras.layers.Dense(10, activation='softmax')   # Output layer (10 classes)
])

In [45]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',   # since y = integer labels
    metrics=['accuracy']
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop]
)

Epoch 1/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9422 - loss: 0.2284 - val_accuracy: 0.9663 - val_loss: 0.1815
Epoch 2/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9476 - loss: 0.1955 - val_accuracy: 0.9663 - val_loss: 0.1640
Epoch 3/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9609 - loss: 0.1659 - val_accuracy: 0.9663 - val_loss: 0.1578
Epoch 4/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9613 - loss: 0.1581 - val_accuracy: 0.9697 - val_loss: 0.1499
Epoch 5/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9666 - loss: 0.1473 - val_accuracy: 0.9731 - val_loss: 0.1392
Epoch 6/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9642 - loss: 0.1491 - val_accuracy: 0.9764 - val_loss: 0.1324
Epoch 7/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9595 - loss: 0.1691 - val_accuracy: 0.9697 - val_loss: 0.1295
Epoch 8/100
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9673 - loss: 0.1325 - val_accuracy: 0.9596 - v

In [46]:
loss, acc = model.evaluate(test_ds)
print(f"\n Final Test Accuracy: {acc:.4f}")


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9727 - loss: 1.3354

 Final Test Accuracy: 0.9832
